# Análisis de Siniestros Viales en Buenos Aires (2019-2023)

Este notebook documenta el proceso de carga, limpieza y preparación de datos de siniestros viales en la Ciudad de Buenos Aires para análisis posterior.

## 1. Carga de datos y configuración del entorno

In [1]:
import pandas as pd

# Carga de archivos desde la fuente original
# - siniestros_viales_hechos.xlsx: Información sobre los eventos/accidentes

try:
    # Usando rutas relativas para mejor portabilidad
    df_accidents = pd.read_excel("../data/raw/siniestros_viales_hechos.xlsx")
    
    print(f"Datos cargados exitosamente:")
    print(f"- Hechos: {df_accidents.shape[0]} registros, {df_accidents.shape[1]} columnas")
except FileNotFoundError as e:
    print(f"Error al cargar archivos: {e}")
    print("Verifique que los archivos se encuentren en la ruta correcta")

Datos cargados exitosamente:
- Hechos: 44012 registros, 23 columnas


## 2. Exploración inicial y diagnóstico de datos


In [2]:
# Verificar valores nulos en el dataset
print("Cantidad de valores nulos por columna en el dataset de accidentes:")
print(df_accidents.isnull().sum())


# Examinar tipos de datos
print("\nTipos de datos en el dataset de accidentes:")
print(df_accidents.dtypes)

# Resumen estadístico básico para variables numéricas
print("\nResumen estadístico del dataset de accidentes:")
print(df_accidents.describe())

Cantidad de valores nulos por columna en el dataset de accidentes:
id_hecho                     0
n_victimas                   0
fecha                        0
aaaa                         0
mm                           0
dd                           0
hora                         0
hh                           0
direccion_normalizada        0
calle                    10901
altura                   11347
cruce                    14503
otra_direccion              31
comuna                       0
tipo_de_calle                0
geocodificacion_caba         0
longitud                   216
latitud                    216
participantes                0
victima                      0
contraparte                  0
gravedad                     0
tipo_de_dato                 0
dtype: int64

Tipos de datos en el dataset de accidentes:
id_hecho                         object
n_victimas                        int64
fecha                    datetime64[ns]
aaaa                              int64
mm

In [3]:
# Función para mostrar valores únicos limitando la cantidad para mejor visualización
def show_unique_values(df, max_items=10):
    """
    Muestra los valores únicos de cada columna en un DataFrame
    
    Parámetros:
    df (pandas.DataFrame): DataFrame a explorar
    max_items (int): Número máximo de valores únicos a mostrar por columna
    """
    for column in df.columns:
        unique_values = df[column].unique()
        print(f"\nValores únicos en {column} ({len(unique_values)} valores):")
        
        # Si hay muchos valores únicos, mostrar solo algunos
        if len(unique_values) > max_items:
            print(unique_values[:max_items], '... y', len(unique_values) - max_items, 'más')
        else:
            print(unique_values)

# Explorar valores únicos en cada dataset
print("Exploración de valores únicos en dataset de accidentes:")
show_unique_values(df_accidents)

Exploración de valores únicos en dataset de accidentes:

Valores únicos en id_hecho (44012 valores):
['LC-2019-0008283' 'LC-2019-0007634' 'LC-2019-0008974' 'LC-2019-0010983'
 'LC-2019-0011092' 'LC-2019-0017186' 'LC-2019-0022256' 'LC-2019-0687058'
 'LC-2019-0688515' 'LC-2019-0757409'] ... y 44002 más

Valores únicos en n_victimas (17 valores):
[ 2  1  4  3  5 13  6  8  9  7] ... y 7 más

Valores únicos en fecha (1826 valores):
<DatetimeArray>
['2019-01-04 00:00:00', '2019-01-05 00:00:00', '2019-01-06 00:00:00',
 '2019-01-07 00:00:00', '2019-01-09 00:00:00', '2019-01-11 00:00:00',
 '2019-01-12 00:00:00', '2019-01-13 00:00:00', '2019-01-15 00:00:00',
 '2019-01-17 00:00:00']
Length: 10, dtype: datetime64[ns] ... y 1816 más

Valores únicos en aaaa (5 valores):
[2019 2020 2021 2022 2023]

Valores únicos en mm (12 valores):
[ 1  2  3  4  5  6  7  8  9 10] ... y 2 más

Valores únicos en dd (31 valores):
[ 4  5  6  7  9 11 12 13 15 17] ... y 21 más

Valores únicos en hora (2607 valores):
[datet

## 4. Limpieza y transformación de datos

### 4.1 Eliminación de columnas innecesarias

In [4]:
# Eliminamos columnas redundantes o que no aportan valor para el análisis

columns_to_drop = [
    "direccion_normalizada", # Redundante
    "cruce",                 # No es necesario para identificar la dirección
    "otra_direccion",        # Información redundante
    "comuna",                # Se puede obtener por geocodificación posteriormente si es necesario
    "tipo_de_calle"          # No contiene suficiente información para el análisis
]

df_accidents.drop(columns=columns_to_drop, inplace=True)
print(f"Columnas eliminadas: {columns_to_drop}")
print(f"Nuevas dimensiones del dataset: {df_accidents.shape}")

Columnas eliminadas: ['direccion_normalizada', 'cruce', 'otra_direccion', 'comuna', 'tipo_de_calle']
Nuevas dimensiones del dataset: (44012, 18)


### 4.2 Normalización de direcciones

En esta sección, implementamos funciones para normalizar y completar las direcciones y las coordenadas a partir de diferentes fuentes de datos.

In [5]:
# Importación de librerías para geocodificación
import requests
import json
import time
from tqdm import tqdm

# Activar barra de progreso para pandas
tqdm.pandas()

def is_valid(value):
    """
    Verifica si un valor es válido (no es nulo ni valor de relleno)
    
    Parámetros:
    value: Valor a verificar
    
    Retorna:
    bool: True si el valor es válido, False en caso contrario
    """
    return not pd.isna(value) and value not in {"", "sd", "SD", "0", 0}

def obtener_calle_altura_from_geocodificacion(geocodificacion, max_retries=10):
    """
    Obtiene la calle y altura a partir de las coordenadas en formato POINT
    usando la API del USIG (Buenos Aires)
    
    Parámetros:
    geocodificacion (str): Coordenadas en formato "POINT(x y)"
    max_retries (int): Número máximo de reintentos en caso de fallo
    
    Retorna:
    str: Dirección normalizada o None si no se pudo obtener
    """
    retries = 0
    while retries < max_retries:
        try:
            if not isinstance(geocodificacion, str) or not geocodificacion.lower().startswith("point"):
                return None
                
            # Extraer coordenadas del formato POINT
            values = geocodificacion.lower().replace("point(", "").replace("point (", "").replace(")", "").replace(",", " ").split()
            if len(values) < 2:
                return None
                
            x, y = values[0], values[1]
            url = f"http://ws.usig.buenosaires.gob.ar/geocoder/2.2/reversegeocoding?x={x}&y={y}"
            
            # Realizar solicitud a la API
            response = requests.get(url, timeout=5)
            
            if response.text[2:-2] == "ErrorCoordenadasErroneas":
                return "ErrorCoordenadasErroneas"
                
            if response.status_code == 200:
                txt = response.text[1:-1]
                data = json.loads(txt)
                return data["puerta"]
                
            return None
            
        except requests.exceptions.Timeout:
            retries += 1
            time.sleep(2)  # Esperar antes de reintentar
        except Exception as e:
            print(f"Error en geocodificación: {e}")
            return None
            
    return None  # Si se agotan los reintentos

def obtener_calle_altura_from_coordinates(lat, lon, max_retries=5):
    """
    Obtiene la calle y altura a partir de coordenadas usando OpenStreetMap
    
    Parámetros:
    lat (float): Latitud
    lon (float): Longitud
    max_retries (int): Número máximo de reintentos en caso de fallo
    
    Retorna:
    str: Dirección normalizada o mensaje de error
    """
    retries = 0
    while retries < max_retries:
        try:
            url = f"https://nominatim.openstreetmap.org/reverse?lat={lat}&lon={lon}&format=json"
            headers = {"User-Agent": "AccidentAnalysis/1.0 (carlosschopfer@gmail.com)"}  # Es importante identificarse
            
            response = requests.get(url, headers=headers, timeout=10)
            data = response.json()
            
            calle = data.get("address", {}).get("road", "")
            altura = data.get("address", {}).get("house_number", "")
            
            if calle:
                return f"{calle} {altura}".strip().upper()
            else:
                return None
                
        except (requests.exceptions.Timeout, requests.exceptions.ConnectionError):
            retries += 1
            time.sleep(2)  # Esperar antes de reintentar
        except Exception as e:
            return f"ERROR: {str(e)}"
            
    return None  # Si se agotan los reintentos

df_accidents["calle_y_altura"] = None  # Inicializar la columna de dirección

# Función principal para obtener calle y altura
def obtener_calle_altura(row, backup_counter=1000):
    """
    Obtiene la calle y altura a partir de los datos disponibles en la fila
    Intenta diferentes fuentes en orden de prioridad
    
    Parámetros:
    row (Series): Fila del DataFrame
    backup_counter (int): Cada cuántas filas se debe realizar un backup
    
    Retorna:
    str: Dirección normalizada o None si no se pudo obtener
    """
    # global i  # Contador para backups periódicos
    
    # Realizar backup periódico
    # if i % backup_counter == 0:
    #     df_accidents.to_csv(f"../data/processed/accidents_backup_{i}.csv", )
        
    try:
        # Si ya existe, mantenerla
        if is_valid(row["calle_y_altura"]):
            return row["calle_y_altura"]
            
        # Si tenemos calle y altura, combinarlas
        if is_valid(row["calle"]) and is_valid(row["altura"]):
            # i += 1
            return f"{row['calle']} {row['altura']}"
            
        # Si tenemos geocodificación, usarla
        if is_valid(row["geocodificacion_caba"]):
            value = obtener_calle_altura_from_geocodificacion(row["geocodificacion_caba"])
            if value != "ErrorCoordenadasErroneas" and value is not None:
                # i += 1
                return value
                
        # Si tenemos coordenadas, usarlas
        if is_valid(row["latitud"]) and is_valid(row["longitud"]):
            # i += 1
            return obtener_calle_altura_from_coordinates(row["latitud"], row["longitud"])
            
        return pd.NA
        
    except Exception as e:
        print(f"Error procesando fila: {e}")
        return pd.NA

# Inicializar contador global
# i = 0

# Aplicar la función a todo el dataset (puede llevar tiempo)
# Descomentar para ejecutar:
df_accidents["calle_y_altura"] = df_accidents.progress_apply(obtener_calle_altura, axis=1)

100%|██████████| 44012/44012 [2:54:58<00:00,  4.19it/s]   


In [6]:
print(obtener_calle_altura(df_accidents.iloc[0]))  # Probar la función con una fila de ejemplo
print(obtener_calle_altura(df_accidents.iloc[117]))  # Probar la función con otra fila de ejemplo
print(obtener_calle_altura(df_accidents.iloc[1016]))  # Probar la función con otra fila de ejemplo

<NA>
SANCHEZ DE LORIA 795
<NA>


In [8]:
def obtener_lat_lon_from_direccion(direccion, max_retries=5):
    """
    Obtiene la latitud y longitud a partir de una dirección usando OpenStreetMap
    
    Parámetros:
    direccion (str): Calle y altura a geocodificar
    max_retries (int): Número máximo de reintentos en caso de fallo

    Retorna:
    tuple: (latitud, longitud) o None si no se pudo obtener
    """
    retries = 0
    while retries < max_retries:
        try:
            url=f"https://nominatim.openstreetmap.org/search?q={direccion}&format=json&addressdetails=1&limit=1"
            headers = {"User-Agent": "AccidentAnalysis/1.0 (carlosschopfer@gmail.com)"}  # Es importante identificarse
            
            response = requests.get(url, headers=headers, timeout=10)
            data = response.json()
            lat=data[0]["lat"]
            lon=data[0]["lon"]
            if lat:
                return lat, lon
            else:
                return pd.NA, pd.NA
        except:
            retries += 1
            time.sleep(2)
    return pd.NA, pd.NA  # Si se agotan los reintentos
def obtener_lat_lon(row, backup_counter=1000):
    """
    Obtiene la latitud y longitud a partir de los datos disponibles en la fila
    Intenta diferentes fuentes en orden de prioridad
    
    Parámetros:
    row (Series): Fila del DataFrame
    backup_counter (int): Cada cuántas filas se debe realizar un backup
    
    Retorna:
    tuple: (latitud, longitud) o None si no se pudo obtener
    """
    # global i  # Contador para backups periódicos
    
    # Realizar backup periódico
    # if i % backup_counter == 0:
    #     df_accidents.to_csv(f"../data/processed/accidents_backup_{i}.csv", index=False)
        
    try:
        # Si ya existe, mantenerla
        if is_valid(row["latitud"]) and is_valid(row["longitud"]):
            return row["latitud"], row["longitud"]
            
        # Si tenemos calle y altura, combinarlas
        if is_valid(row["calle_y_altura"]):
            #i += 1
            return obtener_lat_lon_from_direccion(row["calle_y_altura"])
    except Exception as e:
        #i += 1
        # Manejar errores
        print(f"Error procesando fila {i}: {e}")
        return pd.NA, pd.NA
    
# Inicializar contador global
# i = 0

# Aplicar la función a todo el dataset (puede llevar tiempo)
# Descomentar para ejecutar:
df_accidents[["latitud", "longitud"]] = df_accidents.progress_apply( lambda row: pd.Series(obtener_lat_lon(row)), axis=1)

100%|██████████| 44012/44012 [49:16<00:00, 14.89it/s]  


### 4.3 Conversión de tipos de datos y normalización

In [9]:
import numpy as np

# Manejo de fechas
pd.to_datetime(df_accidents["fecha"], errors="coerce")

# Manejar horas
df_accidents["hh"] = pd.to_numeric(df_accidents["hh"], errors="coerce")
df_accidents["hh"] = df_accidents["hh"].astype("Int64")  # Tipo entero que permite NaN
df_accidents["hh"] = df_accidents["hh"].replace(0, np.nan) # Se reemplazan ceros por NaN, hora 0 se utiliza para indicar que no se conoce la hora del accidente
# Normalizar textos a mayúsculas para consistencia
text_columns = ["calle_y_altura", "victima", "contraparte", "participantes"]
for col in text_columns:
    if col in df_accidents.columns:
        df_accidents[col] = df_accidents[col].astype(str).str.upper()
        # Reemplazar valores no válidos por NaN
        df_accidents[col] = df_accidents[col].replace(["NAN", "NONE", "NONE NONE", "SD", "SD-SD"], np.nan)

print("Conversión de tipos completada")

Conversión de tipos completada


### 4.4 Limpieza específica y correcciones


In [10]:
# Limpiar valores específicos
print("Realizando limpieza de valores específicos...")

# Normalizar 'gravedad'
df_accidents["gravedad"] = df_accidents["gravedad"].str.upper()
df_accidents["gravedad"] = df_accidents["gravedad"].fillna("LEVE")  # Suponiendo que falta = leve

# Corregir casos especiales en direcciones
# Para autopistas
df_accidents["calle_y_altura"] = df_accidents.apply(
    lambda row: f"{row['calle']} Y {row['calle_y_altura']}" 
    if str(row['calle']).upper().startswith("AUTOPISTA") and is_valid(row['calle_y_altura']) 
    else row['calle_y_altura'], 
    axis=1
)

# Eliminar "Y SD" al final de direcciones
df_accidents["calle_y_altura"] = df_accidents["calle_y_altura"].str.replace(" Y SD$", "", regex=True)

print("Limpieza específica completada")

# Tratamiento de valores "SD" a nulos.

df_accidents = df_accidents.replace(["Sd", "sd", "SD", np.nan], pd.NA)

Realizando limpieza de valores específicos...
Limpieza específica completada


### 4.5 Eliminación de registros con datos críticos faltantes

In [11]:
# Identificar columnas críticas que no deben tener valores nulos
critical_columns = ["fecha", "calle_y_altura", "hh", "mm", "dd", "latitud", "longitud"]

# Contar registros antes de la limpieza
records_before = len(df_accidents)

# Eliminar registros sin datos críticos
df_accidents = df_accidents.dropna(subset=critical_columns)

# Contar registros después de la limpieza
records_after = len(df_accidents)
records_removed = records_before - records_after

print(f"Registros antes de la limpieza: {records_before}")
print(f"Registros después de la limpieza: {records_after}")
print(f"Registros eliminados: {records_removed} ({records_removed/records_before:.2%})")

Registros antes de la limpieza: 44012
Registros después de la limpieza: 40428
Registros eliminados: 3584 (8.14%)


## 5. Guardar datos procesados


In [12]:
# Guardar el dataset limpio y procesado
output_path = "../data/processed/accidents_clean.csv"
df_accidents.to_csv(output_path, index=False)

print(f"Dataset limpio guardado en: {output_path}")
print(f"Dimensiones finales: {df_accidents.shape}")

Dataset limpio guardado en: ../data/processed/accidents_clean.csv
Dimensiones finales: (40428, 19)


## 8. Conclusiones y próximos pasos

Este notebook ha realizado la limpieza y preparación inicial de los datos de siniestros viales en Buenos Aires.  

#### Los principales logros son:  

    Normalización de direcciones y geocodificación de ubicaciones.  
    Corrección de tipos de datos y valores inconsistentes.  
    Eliminación de registros incompletos en campos críticos.  

#### Próximos pasos:  

    Análisis exploratorio inicial de distribución temporal y por hora.  
    Visualización espacial preliminar de los accidentes.  
    Análisis más profundo por tipo de participantes.  
    Identificar patrones espaciotemporales y puntos críticos.  
    Correlacionar los accidentes con variables externas (clima, tráfico, etc.).  
    Desarrollar modelos predictivos para identificar factores de riesgo.  
    Generar recomendaciones para políticas de seguridad vial.